In [30]:
import os
import shutil
import glob

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker

from utils import *
from plot_utils import *
from AnalyseDefects_dev import AnalyseDefects

import massPy as mp

In [35]:
def sort_files(files, key = lambda x: int(x.split('frame')[-1].split('.')[0])):
    return sorted(files, key=key)


In [60]:
path = "Z:/nematic_simulation/ns2048pd/output_test_zeta_0.018/output_test_zeta_0.018_counter_0"
outpath = 'data/nigar/ns2048zeta0.018'

frames = os.listdir(path)
frames.remove('parameters.json')
frames_sorted =sort_files(frames)
frames_sorted

if not os.path.exists(outpath):
    os.makedirs(outpath)
# copy params file to outpath
shutil.copyfile(os.path.join(path, 'parameters.json'), os.path.join(outpath, 'parameters.json'))

for i, frame in enumerate(frames_sorted):
    if i < 15:
        file_path = os.path.join(path, frame)
        npz_obj = np.load(file_path)
        vx = npz_obj['vx']
        vy = npz_obj['vy']

        np.savez_compressed(os.path.join(outpath, frame), vx=vx, vy=vy)

In [ ]:
import os
import glob
import numpy as np

def sort_files(files, key = lambda x: int(x.split('frame')[-1].split('.')[0])):
    return sorted(files, key=key)

# set system size
L = 2048

# set directory for frames
frames_dir = outpath

# sort files in directory
files_sorted = sort_files(glob.glob(os.path.join(frames_dir, 'frame*')))

# load velocity fields for first frame
data = np.load(files_sorted[0], allow_pickle=True)
vx = data['vx'].reshape(L, L)
vy = data['vy'].reshape(L, L)
print(vx.shape, vy.shape)

(2048, 2048) (2048, 2048)
